The data is divided in multiple ```csv``` files.<br>
The image below represents the relation between each dataset and it was provided on Kaggle.
<br><br>

<img src = "https://i.imgur.com/HRhd2Y0.png">

<br>

## Business question  : Which products categories are the most prone to customer insatisfaction?

```order_items``` is the table that should give us the connection between an orders and products.
<br><br>
To understand better the relationship between orders, items, products and category, we will first have to explore some of these fields.<br>
For this analysis we will consider and read only the datasets that are relevant for our business question.
<br><br>
We will restrict this analysis to the usage of SQL only, for educational purposes and in order to showcase SQL specific skills.
<br><br>
Therefore we will create a table for each dataset we need.


In [1]:
import pandas as pd
import sqlite3

In [2]:
orders_df = pd.read_csv('../data/raw/olist_orders_dataset.csv')
orders_items_df = pd.read_csv('../data/raw/olist_order_items_dataset.csv')
products_df = pd.read_csv('../data/raw/olist_products_dataset.csv')
customers_df = pd.read_csv('../data/raw/olist_customers_dataset.csv')
reviews_df = pd.read_csv('../data/raw/olist_order_reviews_dataset.csv')
category_translations_df = pd.read_csv('../data/raw/product_category_name_translation.csv')

In [3]:
# Connect to sqlite db and create tables if they don't exist
cnx = sqlite3.connect('olist.db')

orders_df.to_sql(name='orders', con=cnx, if_exists='replace', index=False)
orders_items_df.to_sql(name='orders_items', con=cnx, if_exists='replace', index=False)
products_df.to_sql(name='products', con=cnx, if_exists='replace', index=False)
customers_df.to_sql(name='customers', con=cnx, if_exists='replace', index=False)
reviews_df.to_sql(name='order_reviews', con=cnx, if_exists='replace', index=False)
category_translations_df.to_sql(name='category_translations', con=cnx, if_exists='replace', index=False)

In [4]:
%%capture
%load_ext sql
%sql sqlite:///olist.db

<br>

## Order items exploration
In the description of the dataset on Kaggle we can read the following note:
>"An order might have multiple items."
<br>

Let's validate this assumption by using ```order_item_id``` in order_items table.

In [5]:
%%sql
WITH orders_with_multiple_items AS ( 
    SELECT COUNT(DISTINCT order_item_id) AS items_count, order_id
    FROM orders_items
    GROUP BY order_id
    HAVING items_count>1
    ORDER BY items_count DESC
)
SELECT 
    COUNT(*) AS orders_with_multiple_items_count, 
    ROUND(
        CAST( COUNT(*) AS FLOAT) / ( SELECT COUNT(*) FROM orders) * 100, 3
    ) AS percentage_of_orders,
    MAX(items_count) AS max_items_count
FROM orders_with_multiple_items m
LIMIT 5;

orders_with_multiple_items_count,percentage_of_orders,max_items_count
9803,9.858,21


So it looks like there are multiple orders that have multiple items.<br>
In the documentation, ```order_item_id``` is described as : 

>sequential number identifying number of items included in the same order.

<br>


## Products exploration

Our business question is about the **category** that has the best/worst ratings.<br>
Looking at the products table, we can see that category is a feature of a product, so we will have to focus on ```product_id``` column in ```order_items``` table.

In [6]:
%%sql
WITH orders_with_multiple_prods AS ( 
    SELECT COUNT(DISTINCT product_id) AS prods_count, order_id
    FROM orders_items
    GROUP BY order_id
    HAVING prods_count>1
    ORDER BY prods_count DESC
)
SELECT 
    COUNT(*) AS orders_with_multiple_prods_count,
    ROUND(
       CAST( COUNT(*) AS FLOAT) / ( SELECT COUNT(*) FROM orders) * 100, 3
    ) AS percentage_of_orders,
    MAX(prods_count) AS max_prods_count
FROM orders_with_multiple_prods m;

orders_with_multiple_prods_count,percentage_of_orders,max_prods_count
3236,3.254,8


A little more than **3%** of orders have multiple products and the highest amount of products is **8**.
<br><br>
The only information about reviews we have is located in the ```order_reviews table```, where each review is referring to a specific order, not to a product.
<br><br>
Therefore, to find the category with the worst/best ratings, we would have to look at products in each order. <br>
Orders with multiple products complicate the analysis because we can't easily find, for a given order review, which product contributed to it and with which weight.
<br><br>
Since we found out that just about **3%** of orders have multiple products, we are going to exclude them from the order reviews analysis.

<br>

## Reviews exploration

In [7]:
%%sql
SELECT COUNT(DISTINCT review_id) AS unique_review_id,
    COUNT(DISTINCT order_id) AS unique_order_id
FROM order_reviews;

unique_review_id,unique_order_id
98410,98673


The number of unique ```review_id``` and ```order_id``` don't match.<br>
We are going to look for duplicate ```order_id``` or ```review_id```.
<br><br>

## Duplicate review_id exploration 

In [8]:
%%sql
SELECT order_id, COUNT( DISTINCT review_id) AS review_id_count
FROM order_reviews
GROUP BY order_id
HAVING COUNT(review_id) > 1
ORDER BY COUNT(review_id) DESC
LIMIT 3;

order_id,review_id_count
df56136b8031ecd28e200bb18e6ddb2e,3
c88b1d1b157a9999ce368f218a407141,3
8e17072ec97ce29f0e1f111e598b0c85,3


It looks like there are multiple reviews for the same order.
Let's look at one example with 3 reviews.

In [9]:
%%sql
SELECT * FROM reviews WHERE order_id="8e17072ec97ce29f0e1f111e598b0c85"

review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
67c2557eb0bd72e3ece1e03477c9dff5,8e17072ec97ce29f0e1f111e598b0c85,1,None,Entregou o produto errado.,2018-04-07 00:00:00,2018-04-08 22:48:27
2d6ac45f859465b5c185274a1c929637,8e17072ec97ce29f0e1f111e598b0c85,1,None,Comprei 3 unidades do produto vieram 2 unidades que não corresponde com o que comprei. Devido a minha opinião é negativa com relação a esse vendedor pois não não cumpriu com o prometido na venda.,2018-04-07 00:00:00,2018-04-07 21:13:05
6e4c4086d9611ae4cc0cc65a262751fe,8e17072ec97ce29f0e1f111e598b0c85,1,None,"Embora tenha entregue dentro do prazo, não enviou o produto que comprei.",2018-04-14 00:00:00,2018-04-16 11:37:31


These reviews have a different ```comment``` and ```review_answer_timestamp```. <br><br>

```review_answer_timestamp``` : 
>Shows satisfaction survey answer timestamp.

<br>

There are also two reviews with the same ```review_creation_date```.<br>

```review_creation_date``` :
>Shows the date in which the satisfaction survey was sent to the customer.

<br>
These two reviews' answers were submitted at a different time ( even different day ), so it looks like, unless we have data quality issues, that the user was able to submit answers twice for the same survey.
<br><br>

But how often does it happen that an order has more than one review and ```review_answer_timestamp``` is different? 

In [10]:
%%sql
WITH multiple_reviews_orders AS (
    SELECT order_id, COUNT(review_id) 
    FROM order_reviews
    GROUP BY order_id
    HAVING COUNT(review_id) > 1 AND COUNT(DISTINCT review_answer_timestamp)>1
    ORDER BY COUNT(DISTINCT review_id) DESC
)
SELECT COUNT(*) AS orders_with_multiple_reviews_answers,
       ROUND(
        CAST( COUNT(*) AS FLOAT) / ( SELECT COUNT(DISTINCT order_id) FROM reviews) * 100, 3
       ) AS percentage_of_reviewed_orders
FROM multiple_reviews_orders;

orders_with_multiple_reviews_answers,percentage_of_reviewed_orders
547,0.554


And how often does an order have more than one review and review_creation_date is different?

In [11]:
%%sql
WITH multiple_reviews_orders AS (
    SELECT order_id, COUNT(review_id) 
    FROM order_reviews
    GROUP BY order_id
    HAVING COUNT(review_id) > 1 AND COUNT(DISTINCT review_creation_date)>1
    ORDER BY COUNT(order_id) DESC
)
SELECT COUNT(*) AS orders_with_multiple_reviews_surveys,
       ROUND(
        CAST( COUNT(*) AS FLOAT) / ( SELECT COUNT(DISTINCT order_id) FROM reviews) * 100, 3
       ) AS percentage_of_reviewed_orders
FROM multiple_reviews_orders;

orders_with_multiple_reviews_surveys,percentage_of_reviewed_orders
392,0.397


There are a few of these cases, but the percentage is not that high.
<br><br>
Looking at the **reviews dataset** documentation on Kaggle, we can read the following : 
>This dataset includes data about the reviews made by the customers.
After a customer purchases the product from Olist Store a seller gets notified to fulfill that order.<br> Once the customer receives the product, or the estimated delivery date is due, the customer gets a satisfaction survey by email where he can give a note for the purchase experience and write down some comments.

<br>
From this information it's not very clear how it can happen that an user would get more than one survey for the same order. <br>
Regarding the user being able to submit more than one answer for the same survey, we can only guess that the software used to collect reviews doesn't prevent multiple submissions.
<br><br>

## Duplicate order_id exploration 

In [12]:
%%sql
SELECT COUNT(order_id) - COUNT(DISTINCT order_id) 
FROM order_reviews;

COUNT(order_id) - COUNT(DISTINCT order_id)
551


There are some rows with duplicated ```order_id```.<br>
So probably some reviews are linked to different orders. <br><br>
Let's see how many :

In [13]:
%%sql
WITH review_for_multiple_orders AS (
    SELECT review_id, COUNT(DISTINCT order_id) 
    FROM reviews
    GROUP BY review_id
    HAVING COUNT(DISTINCT order_id) > 1
    ORDER BY COUNT(DISTINCT order_id) DESC
)
SELECT COUNT(*) AS reviews_for_multiple_orders_count,
       ROUND(
        CAST( COUNT(*) AS FLOAT) / ( SELECT COUNT(DISTINCT review_id) FROM reviews) * 100, 3
       ) AS percentage_of_reviews_for_multiple_orders
FROM review_for_multiple_orders;

reviews_for_multiple_orders_count,percentage_of_reviews_for_multiple_orders
789,0.802


According to the dataset documentation, the customer receives a survey after each order is finalised, therefore this is not expected.<br><br>
Let's look closely at the review that is linked to the most amount of orders.<br>
We will select a review that has a comment message.

In [14]:
%%sql
SELECT review_id, COUNT(DISTINCT order_id) AS order_id_count
FROM order_reviews
GROUP BY review_id
HAVING COUNT(order_id) > 1 AND review_comment_message IS NOT NULL
ORDER BY COUNT(order_id) DESC
LIMIT 1;

review_id,order_id_count
dbdf1ea31790c8ecfcc6750525661a9b,3


In [15]:
%%sql
SELECT * FROM reviews WHERE review_id="dbdf1ea31790c8ecfcc6750525661a9b"

review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
dbdf1ea31790c8ecfcc6750525661a9b,fa06c9f04ef55a2a43f2246a0777b7c8,1,None,"O produto veio embalado apenas por um frágil envelope, o que ocasionou o fato da caixa do cartucho ficar totalmente destruída.",2018-03-20 00:00:00,2018-03-21 01:02:05
dbdf1ea31790c8ecfcc6750525661a9b,3cf387bb14e9db171ccbb9b87ea607bb,1,None,"O produto veio embalado apenas por um frágil envelope, o que ocasionou o fato da caixa do cartucho ficar totalmente destruída.",2018-03-20 00:00:00,2018-03-21 01:02:05
dbdf1ea31790c8ecfcc6750525661a9b,9406240a4e41945ba492020a2702f757,1,None,"O produto veio embalado apenas por um frágil envelope, o que ocasionou o fato da caixa do cartucho ficar totalmente destruída.",2018-03-20 00:00:00,2018-03-21 01:02:05


According to this example reviews for multiple order ids are identical (all other columns have same values).<br>
Let's see if this is the case for every review with duplicated ```order_id```.<br>

In [16]:
%%sql
WITH identical_reviews_diff_orders AS
(
    SELECT COUNT(*), review_id
    FROM order_reviews
    GROUP BY review_id, 
            review_score, 
            review_comment_title,
            review_comment_message, review_creation_date,review_answer_timestamp
    HAVING COUNT(DISTINCT order_id)>1
)
SELECT COUNT(*)
FROM identical_reviews_diff_orders;


COUNT(*)
789


The number of reviews linked to multiple orders, where all the other columns are identical, is **789**, which exactly matches the overall number of reviews with multiple orders.
<br><br>
This seems to be a data quality issue, and it might be confirmed by [this open issue](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce/discussion/71650?select=olist_order_reviews_dataset.csv&search=review_id) on Kaggle.
<br><br>
Since the issue is still not resolved, we will simply just ignore those reviews for our case study analysis. <br>
In a real world scenario we might have handled the issue in different ways, depending on the root cause. <br><br>

## Product Category exploration 

In [17]:
%%sql
SELECT COUNT(*) AS prods_without_category,
        ROUND(
            CAST( COUNT(*) AS FLOAT) / ( SELECT COUNT(DISTINCT product_id) FROM products) * 100, 3
        ) AS percentage
FROM products 
WHERE product_category_name IS NULL;

prods_without_category,percentage
610,1.851


We have a small percentage of products that are not assigned to any category.
<br><br>

## Summary of what we found out :

- About **3%** of orders have different kind of products. This is normal and expected, but we will ignore them to simplify the analysis about categories.
- An order can have multiple reviews. This wasn't clear after reading the documentation but we can take into account every review for an order.
- A review can refer to different orders. This wasn't expected and it's probably related to a data quality issue. We will ignore these reviews.
<br><br>

## Product review score - Bayesian average
To rank products depending on ratings, we can't simply rely on the mean of ratings' scores, but we want to take into account also **how many** ratings were submitted for each product.<br>

We will use the **Bayesian average** for this purpose : <br><br>


```Bayesian Average = ( product_avg_review_score * product_reviews_count + C * m ) / ( product_reviews_count + m )```

```C``` = average rating of ALL products <br><br>

```m``` = **50** = Arbitrary confidence number <br><br>

If a product has more than **50** ratings, then it will use its true rating. <br>
If it doesn't have enough ratings, then the missing ones will be counted as the average score of all ratings given to ALL products.<br>
For an item that has more than the threshold (```m```) amount of votes, the Bayesian average doesn’t change its rating average by a significant amount.

In [18]:
%%sql
DROP VIEW IF EXISTS orders_with_single_prods;
CREATE VIEW orders_with_single_prods
AS
SELECT order_id,product_id
    FROM orders_items
    GROUP BY order_id
    HAVING COUNT(DISTINCT product_id)==1;

[]

In [19]:
%%sql
DROP VIEW IF EXISTS reviews_for_single_orders;
CREATE VIEW reviews_for_single_orders
AS
SELECT *, 
        COUNT(DISTINCT order_id), 
        COUNT(*) OVER() AS all_reviews_count,
        AVG(review_score) OVER() AS all_reviews_avg
FROM order_reviews
GROUP BY review_id
HAVING COUNT(DISTINCT order_id)==1

[]

In [20]:
%%sql
DROP VIEW IF EXISTS products_reviews;
CREATE VIEW products_reviews 
AS
SELECT  p.product_id,
        p.product_category_name,
        AVG(o_r.review_score) AS avg_review_score,
        COUNT(*) AS product_reviews_count,
        (AVG(o_r.review_score)*COUNT(*) + o_r.all_reviews_avg*50 )
        / ( COUNT(*) + 50 ) AS bayes_avg,
        o_r.all_reviews_avg
FROM orders_with_single_prods o_i
    JOIN products p ON o_i.product_id=p.product_id
    JOIN reviews_for_single_orders o_r ON o_r.order_id=o_i.order_id
GROUP BY p.product_id;     

[]

### Top 5 products with best reviews

In [21]:
%%sql
SELECT * 
FROM products_reviews 
ORDER BY bayes_avg
LIMIT 5;

product_id,product_category_name,avg_review_score,product_reviews_count,bayes_avg,all_reviews_avg
89b121bee266dcd25688a1ba72eefb61,informatica_acessorios,2.2666666666666666,60,3.0959765549700093,4.0911484209340205
b1d207586fca400a2370d50a9ba1da98,None,2.0232558139534884,43,3.1350260327602264,4.0911484209340205
cd46a885543f0e169a49f1eb25c04e43,informatica_acessorios,1.6071428571428572,28,3.1994541159833463,4.0911484209340205
b5e13c9a353102f79c6206ff5cb61a50,brinquedos,2.9615384615384617,78,3.402792351927352,4.0911484209340205
25c38557cf793876c5abdd5931f922db,bebes,2.611111111111111,36,3.4715979191476865,4.0911484209340205


### Top 5 products with worst reviews

In [22]:
%%sql
SELECT * 
FROM products_reviews 
ORDER BY bayes_avg DESC
LIMIT 5;

product_id,product_category_name,avg_review_score,product_reviews_count,bayes_avg,all_reviews_avg
73326828aa5efe1ba096223de496f596,alimentos,4.846153846153846,52,4.4760531475166765,4.0911484209340205
6a8631b72a2f8729b91514db87e771c0,eletronicos,4.7368421052631575,57,4.435116084548608,4.0911484209340205
e0cf79767c5b016251fe139915c59a26,beleza_saude,4.567796610169491,118,4.425937030039887,4.0911484209340205
363218ba55c610b750224f90bdd34be1,cama_mesa_banho,4.583333333333333,72,4.381618205300828,4.0911484209340205
3e4176d545618ed02f382a3057de32b4,malas_acessorios,4.958333333333333,24,4.3723975817121765,4.0911484209340205


## Category review score - Bayesian average
 
```Bayes Average = ( category_avg_review_score * category_reviews_count + C * m ) / ( category_reviews_count + m )```
<br><br>

```category_avg_review_score``` = the average score of all reviews of all products belonging to a given category
<br><br>

```category_reviews_count``` = the count of all reviews of all products belonging to a given category
<br><br>

```C``` = average rating of ALL reviews = ```4.0911484209340205``` <br><br>

```m``` = **200** -> Arbitrary confidence number <br><br>


In [23]:
%%sql
DROP VIEW IF EXISTS category_reviews_scores;
CREATE VIEW category_reviews_scores 
AS
SELECT p.product_category_name,
       t.product_category_name_english,
       COUNT(DISTINCT review_id) AS category_reviews_count,
       AVG(o_r.review_score) AS category_reviews_avg, 
       (AVG(o_r.review_score)*COUNT(DISTINCT review_id) + 4.0911484209340205*200 ) 
       / ( COUNT(DISTINCT review_id) + 200 ) AS category_bayes_avg
FROM orders_with_single_prods o_i
     JOIN products p ON o_i.product_id=p.product_id
     JOIN reviews_for_single_orders o_r ON o_r.order_id=o_i.order_id
     LEFT JOIN category_translations t ON t.product_category_name=p.product_category_name
GROUP BY p.product_category_name;


[]

### Top 5 categories with best reviews

In [24]:
%%sql
SELECT * 
FROM category_reviews_scores 
ORDER BY category_bayes_avg DESC
LIMIT 5;

product_category_name,product_category_name_english,category_reviews_count,category_reviews_avg,category_bayes_avg
livros_interesse_geral,books_general_interest,487,4.478439425051334,4.365690952236979
malas_acessorios,luggage_accessories,997,4.338014042126379,4.296766653455976
livros_tecnicos,books_technical,248,4.44758064516129,4.2884591164884025
papelaria,stationery,2183,4.279890059551077,4.264049384887455
pet_shop,pet_shop,1627,4.284572833435772,4.263398841919433


### Top 5 products with worst reviews

In [25]:
%%sql
SELECT * 
FROM category_reviews_scores 
ORDER BY category_bayes_avg
LIMIT 5;

product_category_name,product_category_name_english,category_reviews_count,category_reviews_avg,category_bayes_avg
moveis_escritorio,office_furniture,1207,3.663628831814416,3.724399206955795
audio,audio,341,3.8475073313782993,3.9375779744672905
None,None,1361,3.9581190301249083,3.975163154507882
fashion_roupa_masculina,fashion_male_clothing,99,3.7777777777777777,3.987390248116402
telefonia_fixa,fixed_telephony,212,3.891509433962264,3.9884215635602045


It looks like office furniture products tend to have the worst ratings.<br><br>
Also, we have a considerable amount of reviews for products that don't have any category information ( ```None``` ). <br>
We can remove

In [26]:
%%sql
SELECT * 
FROM category_reviews_scores 
WHERE product_category_name IS NOT NULL
ORDER BY category_bayes_avg
LIMIT 5;

product_category_name,product_category_name_english,category_reviews_count,category_reviews_avg,category_bayes_avg
moveis_escritorio,office_furniture,1207,3.663628831814416,3.724399206955795
audio,audio,341,3.8475073313782993,3.9375779744672905
fashion_roupa_masculina,fashion_male_clothing,99,3.7777777777777777,3.987390248116402
telefonia_fixa,fixed_telephony,212,3.891509433962264,3.9884215635602045
construcao_ferramentas_seguranca,construction_tools_safety,157,3.859872611464968,3.9894388912795637
